In [1]:
from ultralytics import YOLO
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import shutil
from glob import glob

In [2]:
base_path = '../../dataset/axial'
train_image = os.path.join(base_path, 'images/train')
train_label = os.path.join(base_path, 'labels/train')
test_image = os.path.join(base_path, 'images/test')
test_label = os.path.join(base_path, 'labels/test')
val_image = os.path.join(base_path, 'images/val')
val_label = os.path.join(base_path, 'labels/val')

In [3]:
# seed 및 GPU 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# Read and sort the file list to ensure consistent order
file_list = sorted(os.listdir(train_image))
random_file = random.choice(file_list).split('.')[0]
print("Image :", random_file)

# Read labels from the file
with open(os.path.join(train_label, f'{random_file}.txt'), 'r') as f:
    labels = f.readlines()
    labels = labels[0].split(' ')

# Read and process the image
img = cv2.imread(os.path.join(train_image, f'{random_file}.jpg'), cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Image : 00299_101


In [4]:
# 파일 개수 세는 함수 = 296 + 75 = 371
def count_files(directory):
    return len(os.listdir(directory))

num_train_images = count_files(train_image)
num_test_images = count_files(test_image)
num_train_labels = count_files(train_label)
num_test_labels = count_files(test_label)
num_val_images = count_files(val_image)
num_val_labels = count_files(val_label)
print(f'Train images: {num_train_images}, Train labels: {num_train_labels}')
print(f'Val images: {num_val_images}, Test labels: {num_val_labels}')
print(f'Test images: {num_test_images}, Test labels: {num_test_labels}\n')

Train images: 237, Train labels: 237
Val images: 59, Test labels: 59
Test images: 75, Test labels: 75



In [15]:
#v8n
experiment_name = 'train'
model_size = 'v8n'
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_n = YOLO('yolov8n.pt')
results = model_n.train(
    data='axial_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
    patience=0
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=axial_t1wce_2_class.yaml, epochs=200, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8n, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8n/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8n
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      4.56G      1.589      4.543      1.423         37        640:  93%|█████████▎| 14/15 [00:01<00:00,  9.39it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      4.56G      1.574      4.509      1.404         17        640: 100%|██████████| 15/15 [00:01<00:00,  7.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.19it/s]

                   all         59         62    0.00273      0.792     0.0698     0.0423


...

    200/200      4.63G     0.3439     0.2297     0.8086         14        640: 100%|██████████| 15/15 [00:01<00:00, 13.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 10.72it/s]

                   all         59         62      0.895       0.89      0.961      0.745



200 epochs completed in 0.154 hours.
Optimizer stripped from runs/train/train_v8n/weights/last.pt, 6.3MB
Optimizer stripped from runs/train/train_v8n/weights/best.pt, 6.3MB

Validating runs/train/train_v8n/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.60it/s]


                   all         59         62      0.938      0.897      0.967      0.759
              negative         59         20      0.902        0.9      0.955      0.778
              positive         59         42      0.974      0.893       0.98       0.74
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/train_v8n


In [17]:
result_n = model_n.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.52it/s]


                   all         59         62      0.938      0.897      0.967      0.757
              negative         59         20      0.902        0.9      0.954      0.777
              positive         59         42      0.974      0.895       0.98      0.737
Speed: 0.1ms preprocess, 7.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/train_v8n


In [20]:
experiment_name = 'train'
model_size = 'v8s'
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_s = YOLO('yolov8s.pt')
results = model_s.train(
    data='axial_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
    patience=0
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=axial_t1wce_2_class.yaml, epochs=200, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8s, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8s/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8s
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      6.16G       1.82      5.288      1.598         17        640: 100%|██████████| 15/15 [00:01<00:00,  7.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.94it/s]


                   all         59         62      0.405      0.185      0.144     0.0856

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      4.65G     0.2916     0.1936     0.7964         14        640: 100%|██████████| 15/15 [00:01<00:00,  9.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.31it/s]

                   all         59         62      0.878      0.914      0.933       0.71



200 epochs completed in 0.201 hours.
Optimizer stripped from runs/train/train_v8s/weights/last.pt, 22.5MB
Optimizer stripped from runs/train/train_v8s/weights/best.pt, 22.5MB

Validating runs/train/train_v8s/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.90it/s]


                   all         59         62      0.924      0.889      0.942      0.732
              negative         59         20      0.944       0.85      0.919      0.716
              positive         59         42      0.903      0.929      0.965      0.749
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/train_v8s


In [21]:
result_s = model_s.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.33it/s]


                   all         59         62      0.924      0.889      0.942      0.733
              negative         59         20      0.944      0.849      0.919      0.716
              positive         59         42      0.903      0.929      0.965      0.751
Speed: 0.2ms preprocess, 13.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/train_v8s


In [22]:
experiment_name = 'train'
model_size = 'v8m'
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_m = YOLO('yolov8m.pt')
results = model_m.train(
    data='axial_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
    patience=0
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 49.7M/49.7M [00:04<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=axial_t1wce_2_class.yaml, epochs=200, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8m, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8m/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8m
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/15 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      7.51G      1.588      4.849      1.488         17        640: 100%|██████████| 15/15 [00:03<00:00,  3.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.47it/s]

...

    200/200      7.54G     0.2502     0.1931     0.8047         14        640: 100%|██████████| 15/15 [00:02<00:00,  5.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.37it/s]

                   all         59         62      0.822      0.842      0.896      0.698



200 epochs completed in 0.290 hours.
Optimizer stripped from runs/train/train_v8m/weights/last.pt, 52.0MB
Optimizer stripped from runs/train/train_v8m/weights/best.pt, 52.0MB

Validating runs/train/train_v8m/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.43it/s]


                   all         59         62      0.849      0.815      0.909      0.714
              negative         59         20      0.782       0.75      0.886      0.723
              positive         59         42      0.915      0.881      0.931      0.704
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/train/train_v8m


In [23]:
result_m = model_m.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


                   all         59         62       0.85      0.815      0.909      0.713
              negative         59         20      0.783       0.75      0.885      0.727
              positive         59         42      0.917      0.881      0.933        0.7
Speed: 0.1ms preprocess, 25.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/train_v8m


In [5]:
experiment_name = 'train'
model_size = 'v8l'
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_l = YOLO('yolov8l.pt')
results = model_l.train(
    data='axial_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
    patience=0,
    verbose=False
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 83.7M/83.7M [00:07<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=axial_t1wce_2_class.yaml, epochs=200, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8l, exist_ok=True, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /home/under1/Detect/dataset/axial/labels/train.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8l/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8l
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/15 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      10.7G       1.62      6.011      1.551         17        640: 100%|██████████| 15/15 [00:05<00:00,  2.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.79it/s]

...

    200/200      10.6G     0.3053     0.2199     0.8169         14        640: 100%|██████████| 15/15 [00:04<00:00,  3.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.89it/s]

                   all         59         62       0.93       0.84      0.912      0.707



200 epochs completed in 0.379 hours.
Optimizer stripped from runs/train/train_v8l/weights/last.pt, 87.7MB
Optimizer stripped from runs/train/train_v8l/weights/best.pt, 87.7MB

Validating runs/train/train_v8l/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 43608150 parameters, 0 gradients, 164.8 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.20it/s]


                   all         59         62      0.961      0.846      0.925      0.723
Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/train/train_v8l


In [6]:
result_l = model_l.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 43608150 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /home/under1/Detect/dataset/axial/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.85it/s]


                   all         59         62      0.962      0.846      0.925      0.724
Speed: 0.3ms preprocess, 17.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/train_v8l
